In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.contrib.learn.python.learn.estimators.estimator import SKCompat
from tensorflow.contrib import rnn
from tensorflow.python.ops import array_ops as array_ops_
learn = tf.contrib.learn

In [2]:
pd_ads=pd.read_csv('D:\\sale\\t_ads.csv')
pd_comment=pd.read_csv('D:\\sale\\t_comment.csv')
pd_order=pd.read_csv('D:\\sale\\t_order.csv')
pd_product=pd.read_csv('D:\\sale\\t_product.csv')
pd_sale=pd.read_csv('D:\\sale\\t_sales_sum.csv')

C:\Users\sjt\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
sale_1036=pd_sale[pd_sale['shop_id']==1036]
comment_1036=pd_comment[pd_comment['shop_id']==1036]
order_1036=pd_order[pd_order['shop_id']==1036]
product_1036=pd_product[pd_product['shop_id']==1036]
ads_1036=pd_ads[pd_ads['shop_id']==1036]

sale_1036['dt']=pd.to_datetime(sale_1036['dt'])
comment_1036['create_dt']=pd.to_datetime(comment_1036['create_dt'])
order_1036['ord_dt']=pd.to_datetime(order_1036['ord_dt'])
product_1036['on_dt']=pd.to_datetime(product_1036['on_dt'])
ads_1036['create_dt']=pd.to_datetime(ads_1036['create_dt'])

sale_1036=sale_1036.sort_values(['dt'],ascending=True)
comment_1036=comment_1036.sort_values(['create_dt'],ascending=True)
order_1036=order_1036.sort_values(['ord_dt'],ascending=True)
product_1036=product_1036.sort_values(['on_dt'],ascending=True)
ads_1036=ads_1036.sort_values(['create_dt'],ascending=True)

sale_1036.to_csv('D:\\sale\\sale_1036.csv')
comment_1036.to_csv('D:\\sale\\comment_1036.csv')
order_1036.to_csv('D:\\sale\\order_1036.csv')
product_1036.to_csv('D:\\sale\\product_1036.csv')
ads_1036.to_csv('D:\\sale\\ads_1036.csv')

C:\Users\sjt\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\sjt\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\sjt\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html

In [4]:
shop_id=pd_sale['shop_id']
shop_id=list(set(shop_id))
sale_list=[]
comment_list=[]
order_list=[]
product_list=[]
ads_list=[]
sale_list=[]

for i in shop_id:
    sale_list.append(pd_sale[pd_sale['shop_id']==i])
    comment_list.append(pd_comment[pd_comment['shop_id']==i])
    order_list.append(pd_order[pd_order['shop_id']==i])
    product_list.append(pd_product[pd_product['shop_id']==i])
    ads_list.append(pd_ads[pd_ads['shop_id']==i])
    
    sale_list[i-1]=sale_list[i-1].sort_values(['dt'],ascending=True)
    comment_list[i-1]=comment_list[i-1].sort_values(['create_dt'],ascending=True)
    order_list[i-1]=order_list[i-1].sort_values(['ord_dt'],ascending=True)
    product_list[i-1]=product_list[i-1].sort_values(['on_dt'],ascending=True)
    ads_list[i-1]=ads_list[i-1].sort_values(['create_dt'],ascending=True)
    
    sale_list[i-1]['dt']=pd.to_datetime(sale_list[i-1]['dt'])
    comment_list[i-1]['create_dt']=pd.to_datetime(comment_list[i-1]['create_dt'])
    order_list[i-1]['ord_dt']=pd.to_datetime(order_list[i-1]['ord_dt'])
    product_list[i-1]['on_dt']=pd.to_datetime(product_list[i-1]['on_dt'])
    ads_list[i-1]['create_dt']=pd.to_datetime(ads_list[i-1]['create_dt'])
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064


In [5]:
for i in range(len(order_list)):
    order_list[i].drop('pid',axis=1,inplace=True)
    order_list[i].drop('shop_id',axis=1,inplace=True)
    order_list[i]=order_list[i].groupby(['ord_dt']).sum()
    
for i in range(len(comment_list)):
    comment_list[i].set_index("create_dt", inplace=True)
    comment_list[i].drop('shop_id',axis=1,inplace=True)
for i in range(len(ads_list)):
    ads_list[i].set_index("create_dt",inplace=True)
    ads_list[i].drop('shop_id',axis=1,inplace=True)
for i in range(len(sale_list)):
    sale_list[i].set_index("dt",inplace=True)
    sale_list[i].drop('shop_id',axis=1,inplace=True)

In [6]:
date_list=[]
for i in shop_id:
    tmp=pd.DataFrame(index=pd.date_range('2016-8-1','2017-4-30'))
    date_list.append(pd.concat([tmp,comment_list[i-1], order_list[i-1],ads_list[i-1]], axis=1, join_axes=[tmp.index]))
    date_list[i-1].fillna(value=0,inplace=True)

In [7]:
order_1036.drop('pid',axis=1,inplace=True)
order_1036=order_1036.groupby(['ord_dt']).sum()

comment_1036.drop('shop_id',axis=1,inplace=True)
order_1036.drop('shop_id',axis=1,inplace=True)
ads_1036.drop('shop_id',axis=1,inplace=True)
sale_1036.drop('shop_id',axis=1,inplace=True)


#order_1036.set_index("ord_dt", inplace=True) order_1036已将ord_dt作为索引
sale_1036.set_index("dt", inplace=True)
comment_1036.set_index("create_dt", inplace=True)
product_1036.set_index("on_dt", inplace=True)
ads_1036.set_index("create_dt", inplace=True)

sale_1036

,sale_amt_3m
dt,
2016-06-30,435382.02
2016-07-31,425521.09
2016-08-31,546715.20
2016-09-30,543089.88
2016-10-31,372576.44
2016-11-30,120428.26
2016-12-31,140046.79
2017-01-31,251676.49


In [8]:
#date_list=pd.date_range('2016-6-1','2017-4-30')
date=pd.DataFrame(index=pd.date_range('2016-8-1','2017-4-30'))
data=pd.concat([date,comment_1036, order_1036,ads_1036], axis=1, join_axes=[date.index])
data.fillna(value=0,inplace=True)
data.to_csv('D:\\sale\\data_1036.csv')
data

,bad_num,cmmt_num,dis_num,good_num,mid_num,sale_amt,offer_amt,offer_cnt,rtn_cnt,rtn_amt,ord_cnt,user_cnt,charge,consume
2016-08-01,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00
2016-08-02,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00
2016-08-03,0.0,0.0,0.0,0.0,0.0,6462.10,2369.94,81.0,2.0,58.12,124.0,124.0,0.00,0.00
2016-08-04,0.0,0.0,0.0,0.0,0.0,5605.59,2290.11,74.0,0.0,0.00,107.0,107.0,0.00,0.00
2016-08-05,0.0,144.0,0.0,144.0,0.0,5328.07,2242.29,72.0,0.0,0.00,104.0,103.0,0.00,0.00
2016-08-06,0.0,198.0,2.0,196.0,2.0,5957.45,2372.53,77.0,0.0,0.00,118.0,118.0,0.00,0.00
2016-08-07,0.0,240.0,0.0,240.0,0.0,5793.08,2289.76,76.0,0.0,0.00,112.0,112.0,0.00,0.00
2016-08-08,0.0,266.0,0.0,266.0,0.0,5626.25,2320.65,75.0,0.0,0.00,109.0,109.0,0.00,0.00
2016-08-09,0.0,360.0,0.0,360.0,0.0,5372.68,2218.75,72.0,0.0,0.00,104.0,104.0,0.00,0.00
2016-08-10,0.0,304.0,0.0,304.0,0.0,5462.70,2204.41,71.0,0.0,0.00,102.0,102.0,0.00,0.00


In [9]:
def norm(df_list):
    for i in range(len(df_list)):
        for j in [col for col in df_list[i]]:
            t_max=df_list[i][j].max()
            t_min=df_list[i][j].min()
            down = 1 if (t_max-t_min == 0) else (t_max-t_min)
            df_list[i][j]=(df_list[i][j]-t_min)*0.1/down
    pass

In [10]:
#data.to_period('M')
data.resample('M').sum()
for i in range(len(date_list)):
    date_list[i]=date_list[i].resample('M').sum()
    
date_list[0].shape
print(np.array(date_list[0],dtype=np.float32))

norm(date_list)

for i in range(len(sale_list)):
    for j in [col for col in sale_list[i]]:
        sale_list[i][j]=pow(sale_list[i][j],0.125)
#norm(sale_list)

[[  0.00000000e+00   2.60000000e+01   0.00000000e+00   2.60000000e+01
    0.00000000e+00   1.45722695e+04   2.39100006e+02   2.40000000e+01
    8.00000000e+00   3.92170013e+02   1.14000000e+02   1.14000000e+02
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   4.08000000e+02   2.00000000e+01   4.07000000e+02
    1.00000000e+00   6.63419062e+04   3.66347998e+03   2.18000000e+02
    1.60000000e+01   9.03049988e+02   5.82000000e+02   5.82000000e+02
    9.57958125e+04   1.01653742e+05]
 [  2.00000000e+00   9.07000000e+02   2.30000000e+01   9.02000000e+02
    3.00000000e+00   1.35478953e+05   5.61570996e+03   4.06000000e+02
    3.90000000e+01   2.47285010e+03   1.15000000e+03   1.14800000e+03
    2.46479199e+04   2.43967891e+04]
 [  6.00000000e+00   7.40000000e+02   2.20000000e+01   7.24000000e+02
    1.00000000e+01   1.60418438e+05   8.96587988e+03   7.06000000e+02
    4.80000000e+01   2.70238989e+03   1.36800000e+03   1.36400000e+03
    0.00000000e+00   0.00000000e+00]
 [  8.000000

In [11]:
date_list[1]

,bad_num,cmmt_num,dis_num,good_num,mid_num,sale_amt,offer_amt,offer_cnt,rtn_cnt,rtn_amt,ord_cnt,user_cnt,charge,consume
2016-08-31,0.03,0.000000,0.018182,0.000000,0.009091,0.000000,0.005087,0.010722,0.008163,0.005792,0.006491,0.006514,0.0,0.0
2016-09-30,0.00,0.005138,0.000000,0.006667,0.000000,0.034758,0.004639,0.004124,0.018367,0.022205,0.036842,0.036796,0.0,0.0
2016-10-31,0.06,0.037549,0.100000,0.033333,0.072727,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.0,0.0
2016-11-30,0.10,0.038340,0.081818,0.031373,0.100000,0.072402,0.097139,0.085361,0.028571,0.025473,0.080000,0.080282,0.0,0.0
2016-12-31,0.10,0.050988,0.031818,0.046667,0.036364,0.036823,0.073192,0.055464,0.048980,0.033574,0.080175,0.080458,0.0,0.0
2017-01-31,0.03,0.008696,0.013636,0.008235,0.018182,0.000120,0.008668,0.012371,0.002041,0.002828,0.000000,0.000000,0.0,0.0
2017-02-28,0.03,0.012648,0.018182,0.010980,0.045455,0.006483,0.000000,0.000000,0.002041,0.000000,0.011930,0.011620,0.0,0.0
2017-03-31,0.06,0.044664,0.031818,0.043137,0.009091,0.028485,0.011893,0.015464,0.014286,0.009712,0.051053,0.050880,0.0,0.0
2017-04-30,0.01,0.100000,0.036364,0.100000,0.009091,0.034883,0.002150,0.002268,0.000000,0.001627,0.071930,0.071831,0.0,0.0


In [12]:
f_num=14
LEARNING_RATE_BASE = 0.01
LEARNING_RATE_DECAY = 0.99
training_steps=150000
HIDDEN_SIZE=30
OUTPUT_SIZE=1

In [13]:
X = tf.placeholder("float",[None,f_num,])
Y = tf.placeholder("float", [None,OUTPUT_SIZE])
global_step = tf.Variable(0, trainable=False)

lay_1 = tf.Variable(tf.random_normal([f_num, HIDDEN_SIZE], mean=0.0, stddev=1.0))
lay_2 = tf.Variable(tf.random_normal([HIDDEN_SIZE,OUTPUT_SIZE],mean=0.0,stddev=1.0))
b_1 = tf.Variable(tf.zeros([HIDDEN_SIZE]),dtype="float32",name="bias_1")
b_2 = tf.Variable(tf.zeros([OUTPUT_SIZE]),dtype="float32",name="bias_2")

lay_out1 = tf.nn.relu(tf.matmul(tf.reshape(X,[-1,f_num]), lay_1) + b_1)

y_=tf.add(tf.matmul(tf.reshape(lay_out1,[-1,HIDDEN_SIZE]),lay_2),b_2)
di=y_-Y
loss_op = tf.reduce_mean(tf.square(y_ - Y))
loss=tf.sqrt(loss_op)
learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        200, LEARNING_RATE_DECAY,
        staircase=True)
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train_op = optimizer.minimize(loss_op,global_step=global_step)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [14]:
with tf.Session() as sess:
    # Run the initializer
    sess.run(init)
    
    for step in range(len(date_list)):
        train_X=np.array(date_list[step],dtype=np.float32)[0:5]
        train_y=np.array(sale_list[step],dtype=np.float32)[2:7]
        sess.run(train_op, feed_dict={X: train_X, Y: train_y})
            
        if step % 10 == 0:
            pass
                #print(sess.run(loss,feed_dict={X:np.array(date_list[0].iloc[5],dtype=np.float32), Y: np.array(sale_list[0].iloc[-1],dtype=np.float32)}))
            print(sess.run([loss,lay_out1],feed_dict={X:train_X, Y: train_y}))
    print(sess.run(y_,feed_dict={X:np.array(date_list[10],dtype=np.float32)[0:5]}))
    result=sess.run(y_,feed_dict={X:np.array(date_list[10],dtype=np.float32)[5:6]})

[3.018611, array([[ 0.02345881,  0.0022358 ,  0.09017844,  0.        ,  0.00900851,
         0.        ,  0.01510095,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.08331991,  0.02375267,
         0.0657263 ,  0.01814973,  0.01194472,  0.06451008,  0.        ,
         0.01941989,  0.00719205,  0.        ,  0.04715344,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.01235513],
       [ 0.032773  ,  0.        ,  0.52726477,  0.38731998,  0.        ,
         0.13625014,  0.11623196,  0.53501064,  0.01941808,  0.        ,
         0.        ,  0.        ,  0.        ,  0.20061854,  0.06490862,
         0.        ,  0.        ,  0.07876263,  0.11537112,  0.        ,
         0.04404463,  0.        ,  0.        ,  0.47411495,  0.        ,
         0.        ,  0.        ,  0.        ,  0.19551662,  0.        ],
       [ 0.07567951,  0.11857671,  0.62304354,  0.4629713 ,  0.        ,
         0.39760777,  0.40719637,  0.4

         0.0641801 ,  0.        ,  0.        ,  0.03536301,  0.18376151]], dtype=float32)]
[0.69541579, array([[ 0.12155683,  0.00740305,  0.46839863,  0.00463206,  0.        ,
         0.07146014,  0.14111315,  0.        ,  0.        ,  0.        ,
         0.28593114,  0.21285266,  0.20527646,  0.35051519,  0.03499828,
         0.45602408,  0.13131493,  0.21125731,  0.33197838,  0.06858052,
         0.07427312,  0.0875515 ,  0.        ,  0.34144324,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.11914799],
       [ 0.13728276,  0.        ,  0.62203544,  0.05669273,  0.0925985 ,
         0.        ,  0.14538924,  0.07620185,  0.00197816,  0.        ,
         0.07080346,  0.        ,  0.22766769,  0.37381417,  0.0735919 ,
         0.48076493,  0.32419604,  0.21318704,  0.69979328,  0.        ,
         0.0628439 ,  0.19324034,  0.        ,  0.36071938,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.1494672 ],
       [ 0.1216885

[0.48157063, array([[  1.72505915e-01,   0.00000000e+00,   5.04624784e-01,
          6.13154620e-02,   0.00000000e+00,   1.12733133e-01,
          2.11246923e-01,   8.91506523e-02,   0.00000000e+00,
          0.00000000e+00,   2.47652069e-01,   4.75859046e-02,
          1.83176592e-01,   1.23213932e-01,   1.85165435e-01,
          5.74982703e-01,   3.68053555e-01,   1.88757256e-01,
          6.71917558e-01,   0.00000000e+00,   1.51032105e-01,
          3.23913991e-03,   0.00000000e+00,   6.22868538e-01,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          2.56068371e-02,   0.00000000e+00,   2.24923134e-01],
       [  1.34038866e-01,   0.00000000e+00,   4.82884765e-01,
          8.80281627e-03,   0.00000000e+00,   0.00000000e+00,
          6.55083805e-02,   0.00000000e+00,   0.00000000e+00,
          5.28033823e-04,   2.35144436e-01,   1.55258030e-01,
          2.33185619e-01,   2.02828482e-01,   8.26330408e-02,
          4.26320970e-01,   3.46461236e-01,   2.52189547

         0.        ,  0.        ,  0.        ,  0.02714977,  0.14863965]], dtype=float32)]
[0.61748618, array([[ 0.1864848 ,  0.07151946,  0.59218657,  0.01895791,  0.        ,
         0.07060508,  0.15217456,  0.        ,  0.        ,  0.        ,
         0.37210351,  0.27931798,  0.20244467,  0.39074871,  0.05365463,
         0.46404573,  0.19077903,  0.1635709 ,  0.43796629,  0.        ,
         0.        ,  0.08816344,  0.        ,  0.48923641,  0.        ,
         0.        ,  0.01953651,  0.        ,  0.        ,  0.18276617],
       [ 0.1934517 ,  0.06124517,  0.67205149,  0.1051054 ,  0.        ,
         0.16184689,  0.19959861,  0.        ,  0.        ,  0.        ,
         0.36854264,  0.18599814,  0.18194239,  0.39854723,  0.05626263,
         0.46738958,  0.17177768,  0.19944073,  0.45418549,  0.        ,
         0.        ,  0.07395341,  0.        ,  0.61668366,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.18207921],
       [ 0.1064651

         0.01313793,  0.        ,  0.        ,  0.        ,  0.27538913]], dtype=float32)]
[0.2638835, array([[ 0.16992578,  0.        ,  0.58194518,  0.        ,  0.        ,
         0.        ,  0.        ,  0.08712295,  0.        ,  0.06220458,
         0.22393005,  0.39787734,  0.10697548,  0.40523496,  0.16976559,
         0.38361943,  0.        ,  0.44804794,  0.33837211,  0.        ,
         0.        ,  0.        ,  0.        ,  0.28776702,  0.        ,
         0.        ,  0.        ,  0.10891271,  0.        ,  0.20672934],
       [ 0.07895621,  0.16578726,  0.56054527,  0.00582061,  0.        ,
         0.1003333 ,  0.27622354,  0.        ,  0.        ,  0.        ,
         0.43743533,  0.24203682,  0.10529567,  0.32315654,  0.05307955,
         0.62081432,  0.23765439,  0.14669763,  0.5591352 ,  0.        ,
         0.        ,  0.17135024,  0.        ,  0.55449504,  0.        ,
         0.        ,  0.0929724 ,  0.        ,  0.        ,  0.15161121],
       [ 0.00639643

...

         0.        ,  0.        ,  0.03858998,  0.        ,  0.2404172 ]], dtype=float32)]
[0.58809537, array([[ 0.04388976,  0.22097468,  0.64477479,  0.16625091,  0.        ,
         0.20845851,  0.33591354,  0.        ,  0.        ,  0.        ,
         0.37625384,  0.15557572,  0.14562429,  0.25944328,  0.18749818,
         0.67566639,  0.33857879,  0.14079517,  0.6762079 ,  0.        ,
         0.        ,  0.09635849,  0.        ,  0.69827074,  0.        ,
         0.        ,  0.03366733,  0.        ,  0.        ,  0.14665043],
       [ 0.08718365,  0.32649356,  0.7186076 ,  0.33490092,  0.        ,
         0.32654703,  0.49269819,  0.        ,  0.        ,  0.        ,
         0.37152848,  0.12625995,  0.06824164,  0.17765689,  0.28395426,
         0.91104734,  0.45829031,  0.06036392,  0.6812526 ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.98706651,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.13498116],
       [ 0.2589796

[0.34976831, array([[ 0.07718714,  0.13160573,  0.91314185,  0.42191228,  0.        ,
         0.40284568,  0.55815011,  0.        ,  0.        ,  0.        ,
         0.2417565 ,  0.        ,  0.09656517,  0.26684836,  0.2160224 ,
         0.84965682,  0.38543078,  0.10355499,  0.80436742,  0.        ,
         0.        ,  0.03493852,  0.        ,  1.00556016,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.10351736],
       [ 0.07516553,  0.        ,  0.7498064 ,  0.15081808,  0.        ,
         0.04872949,  0.22920504,  0.        ,  0.        ,  0.        ,
         0.24319741,  0.00279799,  0.18900406,  0.29098317,  0.07954564,
         0.58154881,  0.34069449,  0.2664021 ,  0.76938438,  0.        ,
         0.        ,  0.15905666,  0.        ,  0.57871568,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.22324465],
       [ 0.27603072,  0.        ,  0.55762738,  0.0745647 ,  0.        ,
         0.        ,  0.08614878,  0

[0.17673947, array([[  1.86167493e-01,   1.88322812e-02,   6.52989149e-01,
          1.22266114e-01,   0.00000000e+00,   0.00000000e+00,
          8.05569142e-02,   0.00000000e+00,   0.00000000e+00,
          6.43179491e-02,   2.93506920e-01,   2.44642749e-01,
          1.56154230e-01,   6.01450086e-01,   1.49465054e-01,
          5.05352259e-01,   3.08239982e-02,   5.98584294e-01,
          3.31936806e-01,   0.00000000e+00,   0.00000000e+00,
          2.42595635e-02,   0.00000000e+00,   3.46916765e-01,
          0.00000000e+00,   0.00000000e+00,   4.26675938e-02,
          0.00000000e+00,   3.75263393e-04,   1.28203526e-01],
       [  1.61849707e-01,   2.58405413e-03,   6.37853861e-01,
          5.59162386e-02,   0.00000000e+00,   0.00000000e+00,
          9.94463861e-02,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   3.32997382e-01,   2.57212907e-01,
          1.65700734e-01,   5.30762255e-01,   1.19596839e-01,
          5.04897892e-01,   7.51731992e-02,   4.17153269

         0.        ,  0.        ,  0.04026263,  0.        ,  0.32850921]], dtype=float32)]
[0.3881866, array([[  2.26552993e-01,   2.40874011e-04,   8.04570675e-01,
          2.74168462e-01,   0.00000000e+00,   0.00000000e+00,
          2.04296038e-01,   8.70406926e-02,   0.00000000e+00,
          0.00000000e+00,   2.83193290e-01,   2.19496399e-01,
          0.00000000e+00,   5.43678701e-01,   9.18138623e-02,
          6.02662146e-01,   9.51554179e-02,   3.20045143e-01,
          3.64627540e-01,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   6.09357238e-01,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   6.21552840e-02,   1.43401355e-01],
       [  2.96940595e-01,   7.44030327e-02,   7.56834090e-01,
          2.40927920e-01,   0.00000000e+00,   0.00000000e+00,
          2.78327703e-01,   9.02112424e-02,   0.00000000e+00,
          0.00000000e+00,   2.65770733e-01,   7.85429776e-02,
          0.00000000e+00,   

[0.378548, array([[ 0.21328187,  0.00173034,  0.59114909,  0.        ,  0.        ,
         0.        ,  0.0471255 ,  0.        ,  0.        ,  0.02593823,
         0.37589943,  0.24835344,  0.20484099,  0.57295674,  0.12288532,
         0.50255555,  0.04700892,  0.42736995,  0.38147235,  0.        ,
         0.        ,  0.0300375 ,  0.        ,  0.23395561,  0.        ,
         0.        ,  0.0152898 ,  0.04206304,  0.        ,  0.21798111],
       [ 0.26765215,  0.        ,  0.63996589,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.03732298,
         0.30583435,  0.1633167 ,  0.19371113,  0.4896552 ,  0.09208523,
         0.46747527,  0.16564828,  0.48563039,  0.51651669,  0.        ,
         0.        ,  0.0569992 ,  0.        ,  0.27908826,  0.        ,
         0.        ,  0.        ,  0.06521407,  0.        ,  0.35026067],
       [ 0.19005716,  0.        ,  0.72117949,  0.0388846 ,  0.        ,
         0.        ,  0.0864757 ,  0. 

In [15]:
date_list[2]

,bad_num,cmmt_num,dis_num,good_num,mid_num,sale_amt,offer_amt,offer_cnt,rtn_cnt,rtn_amt,ord_cnt,user_cnt,charge,consume
2016-08-31,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.007100,0.022346,0.000000,0.000000,0.000000,0.000000,0.047073,0.016481
2016-09-30,0.041667,0.013894,0.024138,0.013183,0.0125,0.022868,0.100000,0.050279,0.023077,0.023562,0.031129,0.031230,0.018812,0.015696
2016-10-31,0.033333,0.043327,0.037931,0.043315,0.0375,0.040645,0.070772,0.069832,0.057692,0.068633,0.045645,0.045631,0.000000,0.000000
2016-11-30,0.050000,0.047532,0.100000,0.046893,0.0625,0.080031,0.099782,0.100000,0.080769,0.082952,0.087258,0.087540,0.015250,0.027852
2016-12-31,0.083333,0.100000,0.058621,0.100000,0.0750,0.100000,0.048673,0.049721,0.057692,0.058868,0.100000,0.100000,0.022830,0.051282
2017-01-31,0.100000,0.036015,0.065517,0.033333,0.1000,0.052247,0.000000,0.000000,0.053846,0.070298,0.049516,0.049676,0.045116,0.100000
2017-02-28,0.016667,0.014991,0.000000,0.013748,0.0875,0.010651,0.003067,0.001862,0.038462,0.044430,0.009355,0.009223,0.014563,0.040308
2017-03-31,0.075000,0.017367,0.031034,0.016008,0.0125,0.031406,0.019622,0.023464,0.100000,0.100000,0.034032,0.034142,0.100000,0.053765
2017-04-30,0.016667,0.010238,0.003448,0.009793,0.0250,0.026380,0.011851,0.013966,0.065385,0.065622,0.028548,0.028641,0.036432,0.070534


In [16]:
sale_list[10]

,sale_amt_3m
dt,
2016-06-30,4.294068
2016-07-31,4.640122
2016-08-31,4.869227
2016-09-30,4.881361
2016-10-31,4.731524
2016-11-30,4.666162
2016-12-31,4.694701
2017-01-31,4.674384
